In [9]:
import getpass
import os

In [15]:
#All Environment Variables

os.environ["LANGSMITH_TRACING"] = "true"

os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
#langsmith_endpoint = os.getenv('LANGSMITH_ENDPOINT') 

os.environ["LANGSMITH_API_KEY"] = "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"
#langsmith_api_key = os.getenv('LANGSMITH_API_KEY') 

os.environ["LANGSMITH_PROJECT"] = "pr-silver-stencil-88"
#langsmith_project = os.getenv('LANGSMITH_PROJECT') 

os.environ["MISTRAL_API_KEY"] = "cL9j05Qnp17FgSxqAlO4N7V17AXdtSVt"

api_key = os.getenv('MISTRAL_API_KEY')


In [11]:
from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [12]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Joy")])

AIMessage(content="Hello Joy! Nice to meet you. How are you today? Let's chat about anything you'd like. 😊", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 35, 'completion_tokens': 26}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-2ee8fae4-a477-4541-bab3-07d55126d7f8-0', usage_metadata={'input_tokens': 9, 'output_tokens': 26, 'total_tokens': 35})

In [13]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have access to personal information about you, including your name. If you'd like, you can tell me your name, and I can use it during our conversation. How would you like me to address you?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 56, 'completion_tokens': 47}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-e4dfaece-489f-4760-9549-9b7f3fd15879-0', usage_metadata={'input_tokens': 9, 'output_tokens': 47, 'total_tokens': 56})

In [14]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Joy"),
        AIMessage(content="Hello Joy! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You just told me your name is Joy! Is that correct?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 28, 'total_tokens': 41, 'completion_tokens': 13}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-b9e30d6c-e2f3-48bc-8e16-fb0a7b3d3e0b-0', usage_metadata={'input_tokens': 28, 'output_tokens': 13, 'total_tokens': 41})

In [16]:
#Message persistence using LangGraph

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [17]:
config = {"configurable": {"thread_id": "abc123"}}

In [18]:
query = "Hi! I'm Joy."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Joy! Nice to meet you. How are you today? Is there something you would like to talk about or do?


In [19]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Joy!


In [20]:
# Changing thread reference leads to fresh conversation history 
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm an artificial intelligence and don't have real-time personal information about you. I can only see and respond to the text you type here. If you'd like me to use a specific name when addressing you, please tell me what it is, and I'll be happy to use it!


In [22]:
# Going back to the original conversation with old thread to get conversation history
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I remember you told me your name is Joy! You just asked me the same question. Do you remember? If you want me to use a different name, just let me know!


In [23]:
# Using Async workflow 
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm an artificial intelligence and don't have real-time personal information about you. I can help you with a wide range of topics, but I don't know your name. If you'd like, you can tell me your name, and I can use it in our conversation!


In [36]:
# Prompt Templates

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [37]:
# Retry now the responses 

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

#Update application state as we have two inputs - messages and language
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [38]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Joy."
language = "Hindi"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

हेल्लो! मैं जॉय से मिलकर खुशी हुई. तुम कैसे हो?


In [39]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

तुम्हारा नाम जॉय है, सही?


In [69]:
# Trim messages from message history to set the context in context window and avoiding overflow.

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=55,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm joy"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm joy", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [70]:
workflow = StateGraph(state_schema=State)

# use trimmed message in prompt
def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": state["messages"], "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [82]:
# Streaming in response to make chat more interactive

config = {"configurable": {"thread_id": "abc567"}}
query = "Which math query I asked?"
language = "Hindi"

input_messages = messages + [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode = "messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")
        

Tumne poocha tha, "2 + 2 kya hota hai?" aur mere se jawab diya gaya tha, "4".

In [74]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Which icecream i like?"
language = "Hindi"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tum vanilla ice cream pasand karate ho.
